In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersCohn
from genomic_benchmarks.models.torch_cnn import CNN
from utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config

## Config

In [10]:
config = {
    "use_padding": False,
    "run_on_gpu": True,
    "dataset": HumanEnhancersCohn,
    "number_of_classes": 1,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
}
check_config(config)

config is correct


## Choose the dataset

In [11]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Unzipping...Done.


## Tokenizer and vocab

In [12]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 7
{'<eos>': 5, 'G': 4, 'A': 3, 'C': 2, '<pad>': 6, 'T': 1, '<bos>': 0}


## Dataloader and batch preparation

In [13]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  500


## Model

In [14]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

## Training

In [15]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0
Train metrics: 
 Accuracy: 65.9%, Avg loss: 0.653561 

Epoch 1
Train metrics: 
 Accuracy: 61.6%, Avg loss: 0.654188 

Epoch 2
Train metrics: 
 Accuracy: 67.7%, Avg loss: 0.649225 

Epoch 3
Train metrics: 
 Accuracy: 69.5%, Avg loss: 0.641505 

Epoch 4
Train metrics: 
 Accuracy: 68.5%, Avg loss: 0.636271 

Epoch 5
Train metrics: 
 Accuracy: 70.7%, Avg loss: 0.630740 

Epoch 6
Train metrics: 
 Accuracy: 70.4%, Avg loss: 0.631523 

Epoch 7
Train metrics: 
 Accuracy: 68.9%, Avg loss: 0.631073 

Epoch 8
Train metrics: 
 Accuracy: 71.0%, Avg loss: 0.633175 

Epoch 9
Train metrics: 
 Accuracy: 72.1%, Avg loss: 0.624318 

Epoch 10
Train metrics: 
 Accuracy: 72.5%, Avg loss: 0.625214 

Epoch 11
Train metrics: 
 Accuracy: 72.7%, Avg loss: 0.630044 

Epoch 12
Train metrics: 
 Accuracy: 72.7%, Avg loss: 0.622582 

Epoch 13
Train metrics: 
 Accuracy: 73.4%, Avg loss: 0.619494 

Epoch 14
Train metrics: 
 Accuracy: 71.8%, Avg loss: 0.619672 



## Testing

In [16]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  140.0959752202034
num_batches 218
correct 4662
size 6948
Test Error: 
 Accuracy: 67.1%, Avg loss: 0.642642 

